### Create table

In [6]:
import sqlalchemy as db
from sqlalchemy import  MetaData, Table, Column, Integer, String, Table, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker




In [2]:
engine = db.create_engine('sqlite:///athletes/athletes.db')
connection = engine.connect()
metadata = db.MetaData()

In [22]:
nbaTeams = Table(
   'NBA_TEAMS', metadata, 
   Column('id', Integer, primary_key = True), 
   Column('abbreviation', String), 
   Column('city', String),
   Column('conference', String),
   Column('division', String),
   Column('full_name', String),
   Column('name', String),
)

In [23]:
metadata.create_all(engine)

### Request data

In [3]:
from pprint import pprint as pp
import requests
import json
response = requests.get('https://www.balldontlie.io/api/v1/teams')

response_json = json.loads(response.content)
print(response_json)

# pp(response.__dict__)
# print(type(response.content))
# pp(response.content)

{'data': [{'id': 1, 'abbreviation': 'ATL', 'city': 'Atlanta', 'conference': 'East', 'division': 'Southeast', 'full_name': 'Atlanta Hawks', 'name': 'Hawks'}, {'id': 2, 'abbreviation': 'BOS', 'city': 'Boston', 'conference': 'East', 'division': 'Atlantic', 'full_name': 'Boston Celtics', 'name': 'Celtics'}, {'id': 3, 'abbreviation': 'BKN', 'city': 'Brooklyn', 'conference': 'East', 'division': 'Atlantic', 'full_name': 'Brooklyn Nets', 'name': 'Nets'}, {'id': 4, 'abbreviation': 'CHA', 'city': 'Charlotte', 'conference': 'East', 'division': 'Southeast', 'full_name': 'Charlotte Hornets', 'name': 'Hornets'}, {'id': 5, 'abbreviation': 'CHI', 'city': 'Chicago', 'conference': 'East', 'division': 'Central', 'full_name': 'Chicago Bulls', 'name': 'Bulls'}, {'id': 6, 'abbreviation': 'CLE', 'city': 'Cleveland', 'conference': 'East', 'division': 'Central', 'full_name': 'Cleveland Cavaliers', 'name': 'Cavaliers'}, {'id': 7, 'abbreviation': 'DAL', 'city': 'Dallas', 'conference': 'West', 'division': 'Southw

In [4]:
pp(response_json['data'][:1])

[{'abbreviation': 'ATL',
  'city': 'Atlanta',
  'conference': 'East',
  'division': 'Southeast',
  'full_name': 'Atlanta Hawks',
  'id': 1,
  'name': 'Hawks'}]


### Create dataframe from respnonse data

In [17]:
import pandas as pd

### Creating a data model

In [7]:
Base = declarative_base()
class NBATeam(Base):
    """NBA team data model."""
    __tablename__ = "NBA_TEAMS"

    id = Column('id', Integer, primary_key = True)
    abbreviation = Column('abbreviation', String) 
    city = Column('city', String)
    conference = Column('conference', String)
    division = Column('division', String)
    full_name = Column('full_name', String)
    name = Column('name', String)
    date_founded = Column('date_founded', DateTime, nullable = True)

In [8]:
Session = sessionmaker(bind=engine)
session = Session()

In [10]:
for i in response_json['data']:
    obj = NBATeam(
        id = i['id'],
        abbreviation = i['abbreviation'],
        city = i['city'],
        conference = i['conference'],
        division = i['division'],
        full_name = i['full_name'],
        name = i['name'])
    # try:
    #     session.add(obj)
    #     session.commit() 
    # except Exception as e:
    #     session.rollback()
    #     print(e)

### Get data from database

In [11]:
nbaTeams = Table('NBA_TEAMS', metadata, autoload=True, autoload_with=engine)

In [12]:
nbaTeams

Table('NBA_TEAMS', MetaData(), Column('id', INTEGER(), table=<NBA_TEAMS>, primary_key=True, nullable=False), Column('abbreviation', VARCHAR(), table=<NBA_TEAMS>), Column('city', VARCHAR(), table=<NBA_TEAMS>), Column('conference', VARCHAR(), table=<NBA_TEAMS>), Column('division', VARCHAR(), table=<NBA_TEAMS>), Column('full_name', VARCHAR(), table=<NBA_TEAMS>), Column('name', VARCHAR(), table=<NBA_TEAMS>), Column('date_founded', DATETIME(), table=<NBA_TEAMS>), schema=None)

In [13]:
query = db.select([nbaTeams]) 
print(query)

SELECT "NBA_TEAMS".id, "NBA_TEAMS".abbreviation, "NBA_TEAMS".city, "NBA_TEAMS".conference, "NBA_TEAMS".division, "NBA_TEAMS".full_name, "NBA_TEAMS".name, "NBA_TEAMS".date_founded 
FROM "NBA_TEAMS"


In [14]:
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
pp(ResultSet)

[(17, 'MIL', 'Milwaukee', 'East', 'ksadlja', 'Milwaukee Bucks', 'asdasd', datetime.datetime(2022, 7, 6, 20, 10, 56, 472000)),
 (18, 'MIN', 'Minnesota', 'West', 'Northwest', 'Minnesota Timberwolves', 'Timberwolves', None),
 (19, 'NOP', 'New Orleans', 'West', 'Southwest', 'New Orleans Pelicans', 'Pelicans', None),
 (20, 'NYK', 'New York', 'East', 'Atlantic', 'New York Knicks', 'Knicks', None),
 (21, 'OKC', 'Oklahoma City', 'West', 'Northwest', 'Oklahoma City Thunder', 'Thunder', datetime.datetime(2022, 7, 6, 20, 42, 42, 486000)),
 (22, 'ORL', 'Orlando', 'East', 'Southeast', 'Orlando Magic', 'Magic', None),
 (23, 'PHI', 'Philadelphia', 'East', 'Atlantic', 'Philadelphia 76ers', '76ers', None),
 (24, 'PHX', 'Phoenix', 'West', 'Pacific', 'Phoenix Suns', 'Suns', None),
 (25, 'POR', 'Portland', 'West', 'Northwest', 'Portland Trail Blazers', 'Trail Blazers', None),
 (26, 'SAC', 'Sacramento', 'West', 'Pacific', 'Sacramento Kings', 'Kings', None),
 (27, 'SAS', 'San Antonio', 'West', 'Southwest', 

### Creating dict from result set

In [15]:
rows = session.query(nbaTeams).all()
list_of_dict = []
for row in rows[:5]:
    list_of_dict.append(row._asdict())
pp(list_of_dict)

[{'abbreviation': 'MIL',
  'city': 'Milwaukee',
  'conference': 'East',
  'date_founded': datetime.datetime(2022, 7, 6, 20, 10, 56, 472000),
  'division': 'ksadlja',
  'full_name': 'Milwaukee Bucks',
  'id': 17,
  'name': 'asdasd'},
 {'abbreviation': 'MIN',
  'city': 'Minnesota',
  'conference': 'West',
  'date_founded': None,
  'division': 'Northwest',
  'full_name': 'Minnesota Timberwolves',
  'id': 18,
  'name': 'Timberwolves'},
 {'abbreviation': 'NOP',
  'city': 'New Orleans',
  'conference': 'West',
  'date_founded': None,
  'division': 'Southwest',
  'full_name': 'New Orleans Pelicans',
  'id': 19,
  'name': 'Pelicans'},
 {'abbreviation': 'NYK',
  'city': 'New York',
  'conference': 'East',
  'date_founded': None,
  'division': 'Atlantic',
  'full_name': 'New York Knicks',
  'id': 20,
  'name': 'Knicks'},
 {'abbreviation': 'OKC',
  'city': 'Oklahoma City',
  'conference': 'West',
  'date_founded': datetime.datetime(2022, 7, 6, 20, 42, 42, 486000),
  'division': 'Northwest',
  'fu

In [ ]:
json_list_of_dict = json.dumps(list_of_dict)
print(type(json_list_of_dict))

### Insert data into DB

In [18]:
# insert data into the NBATeam table
for team_entry in response_json['data']:
    team = NBATeam(
        id = team_entry['id'],
        abbreviation = team_entry['abbreviation'],
        city = team_entry['city'],
        conference = team_entry['conference'],
        division = team_entry['division'],
        full_name = team_entry['full_name'],
        name = team_entry['name'])
    try:
        session.add(team)
        session.commit() 
    except Exception as e:
        session.rollback()
